In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import json

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import torch
from torch.utils.data import Dataset

from transformers import BertModel
from transformers import BertTokenizer
from transformers import BertPreTrainedModel
from transformers.models.bert.modeling_bert import BertEmbeddings, BertEncoder, BertPooler
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions

from typing import *

df_train = pd.read_csv("cola_public\\raw\\in_domain_train.tsv",sep='\t')
df_train.head()

,gj04,1,Unnamed: 2,"Our friends won't buy this analysis, let alone the next one we propose."
0,gj04,1,NaN,One more pseudo generalization and I'm giving up.
1,gj04,1,NaN,One more pseudo generalization or I'm giving up.
2,gj04,1,NaN,"The more we study verbs, the crazier they get."
3,gj04,1,NaN,Day by day the facts are getting murkier.
4,gj04,1,NaN,I'll fix you a drink.


In [14]:
df = df_train.drop(columns=['gj04', 'Unnamed: 2'])
df.columns = ['label', 'context']
df.to_csv('CoLA_train.csv')

In [37]:
class CoLA_Dataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    
    def __getitem__(self, index):
        df = self.df
        EC = self.tokenizer.encode_plus(df['context'][index])

        input_ids = torch.tensor(EC['input_ids'])
        mask = torch.tensor(EC['attention_mask'])
        token = torch.tensor(EC['token_type_ids'])

        return input_ids, mask, token, df['label'][index]

    def __len__(self):
        return len(self.df)
        

In [53]:
def collate_batch(sample): 
    input_ids_batch = [s[0] for s in sample]
    mask_batch = [s[1] for s in sample]
    token_batch = [s[2] for s in sample]
    Label_batch = ([s[3] for s in sample])

    input_ids_batch = pad_sequence(input_ids_batch, batch_first=True)
    mask_batch = pad_sequence(mask_batch, batch_first=True)
    token_batch = pad_sequence(token_batch, batch_first=True)

    return input_ids_batch, mask_batch, token_batch, Label_batch

BATCH_SIZE = 2

In [54]:
train_dataset = CoLA_Dataset(df)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

In [60]:
data = next(iter(train_loader))
data

(tensor([[  101,  1284, 10788,  1111, 10095,  1107,  1103,  1298,   119,   102],
         [  101,  2159,  3885,  1103,  1148,  2259,  1103,  3240,   119,   102]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 [0, 0])